In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_emotion.csv
/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_sentiment_data.csv


In [2]:
# Step 1: Install TensorFlow (if needed)
#!pip install tensorflow

# Step 2: Import Libraries
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle


2025-05-28 14:31:53.586460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748442713.786231      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748442713.847182      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Step 3: Load Dataset
df = pd.read_csv('/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_emotion.csv')
df = df[['sentence', 'emotion']]
df = df.dropna()
df.head()


,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


In [4]:
# Step 4: Clean Text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['cleaned'] = df['sentence'].apply(clean_text)


In [6]:
# Step 5: Label Encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['emotion'])
labels = df['label'].values
num_classes = len(le.classes_)


In [7]:
# Step 6: Tokenization and Padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned'])

sequences = tokenizer.texts_to_sequences(df['cleaned'])
padded = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')


In [8]:
# Step 7: Train/Val/Test Split
X_train, X_temp, y_train, y_temp = train_test_split(padded, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [9]:
# Step 8: Build BiLSTM Model with Dropout
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1748442774.445867      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748442774.446638      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Step 9: Train Model
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    batch_size=128,
    callbacks=[early_stop]
)


Epoch 1/10


I0000 00:00:1748442788.446219     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


2643/2643 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.7508 - loss: 0.6491 - val_accuracy: 0.9306 - val_loss: 0.1292
Epoch 2/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9351 - loss: 0.1273 - val_accuracy: 0.9352 - val_loss: 0.0998
Epoch 3/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9403 - loss: 0.1001 - val_accuracy: 0.9391 - val_loss: 0.0965
Epoch 4/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9416 - loss: 0.0945 - val_accuracy: 0.9402 - val_loss: 0.0936
Epoch 5/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9421 - loss: 0.0885 - val_accuracy: 0.9405 - val_loss: 0.0945
Epoch 6/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9432 - loss: 0.0866 - val_accuracy: 0.9387 - val_loss: 0.0934
Epoch 7/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9441 - loss: 0.0850 - val_accuracy: 0.9397 - val_loss: 0.0962
Epoch 8/10
2643/2643 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.9442 - loss: 0.08

In [11]:
# Step 10: Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_acc:.4f}")


1322/1322 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9400 - loss: 0.0961
✅ Test Accuracy: 0.9399


In [12]:
# Step 11: Save Model and Tokenizer
model.save("sentiment_model.h5")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [14]:
import zipfile

with zipfile.ZipFile('/kaggle/working/model_files.zip', 'w') as zipf:
    zipf.write('sentiment_model.h5')
    zipf.write('tokenizer.pkl')
    zipf.write('label_encoder.pkl')
print("over")


over
